Setup model

In [ ]:
# baseline cnn model for mnist
from numpy import mean
from numpy import std
import numpy as np
import timeit
from matplotlib import pyplot
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
# load dataset
trainX, trainY, test_X, test_Y = load_dataset()
# prepare pixel data
trainX, testX = prep_pixels(trainX, test_X)
# evaluate model
test_X, valX, test_Y, valY = train_test_split(test_X, test_Y, test_size=0.5)
trainX, trainX1, trainY, trainY1 = train_test_split(trainX, trainY, test_size=0.3)
trainX, trainX2, trainY, trainY2 = train_test_split(trainX, trainY, test_size=0.4)
trainX, trainX3, trainY, trainY3 = train_test_split(trainX, trainY, test_size=0.5)
trainX1, trainX4, trainY1, trainY4 = train_test_split(trainX1, trainY1, test_size=0.35)
trainX1, trainX5, trainY1, trainY5 = train_test_split(trainX1, trainY1, test_size=0.25)
trainX2, trainX6, trainY2, trainY6 = train_test_split(trainX2, trainY2, test_size=0.45)
trainX4, trainX7, trainY4, trainY7 = train_test_split(trainX4, trainY4, test_size=0.2)


11501568/11490434 [==============================] - 0s 0us/step


Check shape of input dataset

In [ ]:
print(trainX.shape)
print(trainX1.shape)
print(trainX2.shape)
print(trainX3.shape)
print(trainX4.shape)
print(trainX5.shape)
print(trainX6.shape)
print(trainX7.shape)
print(valX.shape)
print(test_X.shape)


(12600, 28, 28, 1)
(8775, 28, 28, 1)
(9240, 28, 28, 1)
(12600, 28, 28, 1)
(5040, 28, 28, 1)
(2925, 28, 28, 1)
(7560, 28, 28, 1)
(1260, 28, 28, 1)
(5000, 28, 28, 1)
(5000, 28, 28, 1)


Step 1: 
    for 1/10 dataset train for server (sv)
    for 1/10 dataset train for client 1 (c1)
    for 1/10 dataset train for client 2 (c2)
    for 1/10 dataset train for client 3 (c3)

In [ ]:
c1 = define_model()
c2 = define_model()
c3 = define_model()
sv = define_model()

start = timeit.default_timer()
sv.fit(trainX, trainY, epochs=10, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('Server: {}'.format(end-start))

start = timeit.default_timer()
c1.fit(trainX1, trainY1, epochs=10, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C1: {}'.format(end-start))

start = timeit.default_timer()
c2.fit(trainX2, trainY2, epochs=10, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C2: {}'.format(end-start))

start = timeit.default_timer()
c3.fit(trainX3, trainY3, epochs=10, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C3: {}'.format(end-start))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Server: 30.082124149000038
C1: 3.1138961869999093
C2: 1.9538572849999127
C3: 3.094304616999807


Test Step 1: Test model in server and clients (c1, c2, c3)

In [ ]:

print("==============train result=============")
y_pred = sv.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Server: ', accuracy_score(y_pred1, y_pred))

y_pred = c1.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C1: ', accuracy_score(y_pred1, y_pred))

y_pred = c2.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C2: ', accuracy_score(y_pred1, y_pred))

y_pred = c3.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C3: ', accuracy_score(y_pred1, y_pred))
print("========================================")

==============train result=============
Server:  0.9526
C1:  0.9404
C2:  0.9436
C3:  0.9498


Step 2: update weight_avg

In [ ]:
avg = (np.array(c1.get_weights()) + np.array(c2.get_weights()) + np.array(c3.get_weights()) + np.array(sv.get_weights())) / 4
c1.set_weights(avg)
c2.set_weights(avg)
c3.set_weights(avg)
sv.set_weights(avg)
print("==============Result with weight_Avg=============")
y_pred = sv.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Server: ', accuracy_score(y_pred1, y_pred))
y_pred = c1.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Client 1 (c1): ', accuracy_score(y_pred1, y_pred))
print("========================================")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


==============Result with weight_Avg=============
Server:  0.711
Client 1 (c1):  0.711


Step 3: train client 1 and client 2 with new dataset

In [ ]:
start = timeit.default_timer()
c1.fit(trainX4, trainY4, epochs=10, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C1: {}'.format(end-start))

start = timeit.default_timer()
c2.fit(trainX5, trainY5, epochs=30, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C2: {}'.format(end-start))

print("==============train result=============")
y_pred = sv.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Server: ', accuracy_score(y_pred1, y_pred))

y_pred = c1.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C1: ', accuracy_score(y_pred1, y_pred))

y_pred = c2.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C2: ', accuracy_score(y_pred1, y_pred))

y_pred = c3.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C3: ', accuracy_score(y_pred1, y_pred))
print("========================================")

C1: 0.9264401730001737
C2: 2.296632405999844
==============train result=============
Server:  0.711
C1:  0.917
C2:  0.925
C3:  0.711


Step 4: update weight_avg

In [ ]:
avg = (np.array(c1.get_weights()) + np.array(c2.get_weights()) + np.array(c3.get_weights()) + np.array(sv.get_weights())) / 4
c1.set_weights(avg)
c2.set_weights(avg)
c3.set_weights(avg)
sv.set_weights(avg)
print("==============Result Avg=============")
y_pred = sv.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Server: ', accuracy_score(y_pred1, y_pred))
y_pred = c1.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Client 1: ', accuracy_score(y_pred1, y_pred))
print("========================================")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


==============Result Avg=============
Server:  0.917
Client 1:  0.917


Step 5: Next, train client 1 and client 2, no train server and client 3

In [ ]:
start = timeit.default_timer()
c1.fit(trainX6, trainY6, epochs=30, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C1: {}'.format(end-start))

start = timeit.default_timer()
c2.fit(trainX7, trainY7, epochs=10, batch_size=512, validation_data=(valX, valY), verbose=0)
end = timeit.default_timer()
print('C2: {}'.format(end-start))

print("==============Result train=============")
y_pred = sv.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Server: ', accuracy_score(y_pred1, y_pred))

y_pred = c1.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C1: ', accuracy_score(y_pred1, y_pred))

y_pred = c2.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C2: ', accuracy_score(y_pred1, y_pred))

y_pred = c3.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('C3: ', accuracy_score(y_pred1, y_pred))
print("========================================")

C1: 3.330065909000041
C2: 0.6683926630000769
==============Result train=============
Server:  0.917
C1:  0.9538
C2:  0.9156
C3:  0.917


Step 5_3: update weight for all computer and check result

In [ ]:
avg = (np.array(c1.get_weights()) + np.array(c2.get_weights()) + np.array(c3.get_weights()) + np.array(sv.get_weights())) / 4
c1.set_weights(avg)
c2.set_weights(avg)
c3.set_weights(avg)
sv.set_weights(avg)
print("==============result Avg=============")
y_pred = sv.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Server: ', accuracy_score(y_pred1, y_pred))
y_pred = c1.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Client 1: ', accuracy_score(y_pred1, y_pred))
print("========================================")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


==============result Avg=============
Server:  0.9354
Client 1:  0.9354


Step 6: Reuslt when train one computer which full dataset train

In [ ]:
x = np.concatenate((trainX, trainX1, trainX2, trainX3, trainX4, trainX5, trainX6, trainX7), axis=0)
y = np.concatenate((trainY, trainY1, trainY2, trainY3, trainY4, trainY5, trainY6, trainY7), axis=0)
start = timeit.default_timer()
model = define_model()
model.fit(x, y, epochs=30, batch_size=512, validation_data=(valX, valY), verbose=0)
print("==============Result with one computer=============")
y_pred = model.predict(test_X)
y_pred = np.argmax(y_pred, axis=-1)
y_pred1 = np.argmax(test_Y, axis=-1)
print('Result: ', accuracy_score(y_pred1, y_pred))
end = timeit.default_timer()
print('time train full: {}'.format(end-start))
print("========================================")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


==============Result with one computer=============
Result:  0.9792
time train full: 21.372641905000137
